In [30]:
import sqlite3
from werkzeug.security import generate_password_hash, check_password_hash

DB_NAME = 'complaints_management.db'

def get_db_connection():
    return sqlite3.connect(DB_NAME)

def initialize_db(conn):
    cursor = conn.cursor()

    # جدول الشكاوى
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS complaints (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT NOT NULL,
            category TEXT NOT NULL,
            content TEXT NOT NULL,
            priority TEXT NOT NULL,
            status TEXT NOT NULL DEFAULT 'In Progress',
            created_at TEXT NOT NULL
        )
    """)

    # جدول التقييمات
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS feedback (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            complaint_id INTEGER NOT NULL,
            rating INTEGER,
            comments TEXT,
            FOREIGN KEY(complaint_id) REFERENCES complaints(id)
        )
    """)

    # جدول الإدمن
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS admins (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL UNIQUE,
            password TEXT NOT NULL
        )
    """)

    conn.commit()

def authenticate(username, password, conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM admins WHERE username = ?", (username,))
    admin = cursor.fetchone()
    if admin and check_password_hash(admin[2], password):
        return True
    return False

def create_default_admin(conn, username, password):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM admins WHERE username = ?", (username,))
    existing_admin = cursor.fetchone()
    
    if not existing_admin:
        hashed_password = generate_password_hash(password)
        cursor.execute("INSERT INTO admins (username, password) VALUES (?, ?)", (username, hashed_password))
        conn.commit()

def change_password(username, old_password, new_password, conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM admins WHERE username = ?", (username,))
    admin = cursor.fetchone()
    
    if admin and check_password_hash(admin[2], old_password):
        hashed_password = generate_password_hash(new_password)
        cursor.execute("UPDATE admins SET password = ? WHERE username = ?", (hashed_password, username))
        conn.commit()
        return True
    return False

def load_data(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM complaints")
    return cursor.fetchall()


In [32]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('complaints_management.db') 

In [43]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

,name
0,complaints
1,sqlite_sequence
2,feedback
3,admins


In [41]:
df_admins = pd.read_sql("SELECT * FROM admins;", conn)
df_admins

,id,username,password
0,1,admin,scrypt:32768:8:1$tYxbcW4nPv6BmMQh$a7fe472e65de...
1,2,admin,scrypt:32768:8:1$xH2Md5c1su9hewsF$0e1121d9d2e2...
2,3,admin,scrypt:32768:8:1$iFid3WgEHqSskuVO$7d4e42ac54dd...
3,4,admin,scrypt:32768:8:1$VyUhB5hMO3DB6Uhl$ebb395b1efa5...
4,5,admin,scrypt:32768:8:1$Z2ayn0hYNIUMn7Bd$addc4bff80f3...
5,6,admin,scrypt:32768:8:1$jfi0FgdgU6wbWjxX$78e8b9f49cc2...
6,7,admin,scrypt:32768:8:1$k5muMMGJTTt0N8Fn$09e0fa859604...
7,8,admin,scrypt:32768:8:1$IrzjMI3zNJuuLNof$8e73557c786d...
8,9,admin,scrypt:32768:8:1$0xtiE6J9oTbBNHbG$2a66b2807188...
9,10,admin,scrypt:32768:8:1$rMcxXYzSkA1oGmZH$d38a843f65e6...


In [39]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# تنفيذ DROP لكل جدول
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table[0]}")
    conn.commit()